In [1]:
import webbrowser
from IPython.core.display import HTML
import pandas as pd
import nltk
from nltk import ngrams
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import Counter
#import pyLDAvis.gensim
import gensim
import pickle 
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import os
import gensim.corpora as corpora
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess


In [2]:
df = pd.read_csv('../data/ALLrobloxParents.csv')

In [3]:
df.head()

,type,url,title,author,date,timestamp,score,text,comment_id,parent_id
0,Post,https://www.reddit.com/r/RobloxParents/comment...,Can you limit friends?,discovoytng,2022-11-15,1.668478e+09,1,Can you limit friends?\nCan you limit Roblox t...,NaN,NaN
1,comment,https://www.reddit.com/r/RobloxParents/comment...,NaN,zackyattacky,2022-11-15,1.668479e+09,1,You unfortunately can't limit friends or turn ...,iwery2p,NaN
2,comment,https://www.reddit.com/r/RobloxParents/comment...,NaN,discovoytng,2022-11-15,1.668489e+09,2,Thank you!!,iwfc4d5,iwery2p
3,comment,https://www.reddit.com/r/RobloxParents/comment...,NaN,JesseB999,2022-11-15,1.668480e+09,1,It's a terrible platform for little kids. No m...,iwet3lq,iwery2p
4,Post,https://www.forbes.com/sites/danidiplacido/202...,"The ‘Huggy Wuggy’ YouTube Panic, Explained",Morterni,2022-04-29,1.651214e+09,2,"The ‘Huggy Wuggy’ YouTube Panic, Explained\n",NaN,NaN


In [4]:
df['text']

0      Can you limit friends?\nCan you limit Roblox t...
1      You unfortunately can't limit friends or turn ...
2                                            Thank you!!
3      It's a terrible platform for little kids. No m...
4           The ‘Huggy Wuggy’ YouTube Panic, Explained\n
                             ...                        
274    How long has your kid been playing Roblox?\nWo...
275    I’m not a parent but myself have been playing ...
276    Welcoming all parents to the /r/RobloxParents ...
277    Am happy to be a new member of the Roblox Pare...
278    Welcome to the community! all of us here are p...
Name: text, Length: 279, dtype: object

In [5]:
## Key Words

In [6]:
text = " ".join(df['text'].dropna().astype(str))
tokens = nltk.word_tokenize(text)
tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]
stop_words = set(stopwords.words('english'))
filtered_tokens = [token for token in tokens if token.lower() not in stop_words and token.strip() != '']


bi_grams = ngrams(filtered_tokens, 2)

In [7]:
bi_grams_list = list(bi_grams)
bi_grams_list = [tuple(filter(None, bg)) for bg in bi_grams_list]
print(bi_grams_list)

[('limit', 'friends'), ('friends', 'limit'), ('limit', 'Roblox'), ('Roblox', 'certain'), ('certain', 'friends'), ('friends', 'Roblox'), ('Roblox', 'confusing'), ('confusing', '8'), ('8', 'year'), ('year', 'old'), ('old', 'loves'), ('loves', 'one'), ('one', 'friend'), ('friend', 'circle'), ('circle', 'chat'), ('chat', 'app'), ('app', 'message'), ('message', 'turned'), ('turned', 'protect'), ('protect', 'creepy'), ('creepy', 'people'), ('people', 'couple'), ('couple', 'girls'), ('girls', 'tried'), ('tried', 'cyber'), ('cyber', 'bully'), ('bully', 'limit'), ('limit', 'friends'), ('friends', 'Right'), ('Right', 'friend'), ('friend', 'anyone'), ('anyone', 'wants'), ('wants', 'limit'), ('limit', 'chat'), ('chat', 'certain'), ('certain', 'friends'), ('friends', 'Feel'), ('Feel', 'stupid'), ('stupid', 'asking'), ('asking', 'sure'), ('sure', 'amazing'), ('amazing', 'Roblox'), ('Roblox', 'wizards'), ('wizards', 'Thanks'), ('Thanks', 'unfortunately'), ('unfortunately', 'ca'), ('ca', 'nt'), ('nt',

In [8]:
bi_grams_list1 = [' '.join(bi_gram) for bi_gram in bi_grams_list if '(' not in bi_gram and ')' not in bi_gram]

freq_dist = nltk.FreqDist(bi_grams_list1)
most_common = freq_dist.most_common(1)
print('The most common bi-gram is:', most_common[0][0], 'with a frequency of', most_common[0][1])

The most common bi-gram is: parental controls with a frequency of 15


In [9]:
bi_grams_list1 = [' '.join(bi_gram) for bi_gram in bi_grams_list if '(' not in bi_gram and ')' not in bi_gram]

if bi_grams_list1:
    freq_dist = nltk.FreqDist(bi_grams_list1)
    most_common = freq_dist.most_common(1)
    print('The most common bi-gram is:', most_common[0][0], 'with a frequency of', most_common[0][1])
else:
    print('No bigrams found in the text')

The most common bi-gram is: parental controls with a frequency of 15


In [10]:
words = text.split()
words = [word for word in words if word.lower() not in stop_words]
word_counts = Counter(words)

for word, count in word_counts.most_common(10):
    print(f"{word}: {count}")

Roblox: 94
games: 77
game: 72
play: 49
get: 48
playing: 44
kids: 38
chat: 37
kid: 35
Robux: 33


In [11]:
vectorizer = CountVectorizer(ngram_range = (2, 2))
X1=vectorizer.fit_transform(bi_grams_list1)
features = (vectorizer.get_feature_names_out())

vectorizer = TfidfVectorizer(ngram_range = (2, 2))
tfidf = vectorizer.fit_transform(bi_grams_list1)


sums = tfidf.sum(axis = 0)
data1 = []
for col, term in enumerate(features):
    data1.append( (term, sums[0, col] ))
ranking = pd.DataFrame(data1, columns = ['term', 'rank'])
words = (ranking.sort_values('rank', ascending = False))
print ("\n\nWords : \n", words.head(7))



Words : 
                       term  rank
2924     parental controls  18.0
4740              year old  14.0
111   account restrictions  13.0
3087           play roblox  12.0
3147        playing roblox  11.0
1652             get eaten   9.0
4064       started playing   8.0


In [12]:
df_ques = df['text'].str.endswith('?')
df_true = df.loc[df_ques]
print(df_true['text'])

8      A potential solution to lack of parental contr...
17     Filter curated content\nIs there anyway to fil...
27     I don’t really understand what happened, the v...
28     It was like a clip before the game opened (aft...
37     Account Restrictions\nI changed the settings t...
43     Account Hacked and Roblox saw nothing?\nMy son...
44     How long did it take to retain access to the r...
47     What's with all the "get eaten" games?\nI have...
66     Searching for "parental controls"-approved gam...
72     Son getting scammed\nMy 8yo plays Roblox for a...
74                       Was it a game, or avatar items?
83     do the settings described [here](https://en.he...
88     Report it, these games are from around 2014 an...
93                                                     ?
97                                    clickbaited games?
119    What are your biggest safety concerns?\nHi fel...
122         Really? How is that allowed on the platform?
149    My 6yr old is asking me 

In [13]:
df['text_processed'] = df['text'].apply(lambda x: re.sub('[,.!?]', '', x) if isinstance(x, str) else x)
df['text_processed'] = df['text_processed'].apply(lambda x: x.lower() if isinstance(x, str) else x)

df['text_processed'].head()


0    can you limit friends\ncan you limit roblox to...
1    you unfortunately can't limit friends or turn ...
2                                            thank you
3    it's a terrible platform for little kids no ma...
4          the ‘huggy wuggy’ youtube panic explained\n
Name: text_processed, dtype: object

In [14]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = df.text.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

['limit', 'friends', 'limit', 'roblox', 'certain', 'friends', 'roblox', 'confusing', 'year', 'old', 'loves', 'one', 'friend', 'circle', 'chat', 'app', 'message', 'turned', 'protect', 'creepy', 'people', 'couple', 'girls', 'tried', 'cyber', 'bully', 'limit', 'friends', 'right', 'friend']


In [20]:
# generate new file ParentsComments.html and result10

id2word = corpora.Dictionary(data_words)
texts = data_words
corpus = [id2word.doc2bow(text) for text in texts]
num_topics = 10
filename = 'RobloxParents'
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)


LDAvis_data_filepath = os.path.join('../results/'+filename)
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared,  '../results/' + filename + '.html')
LDAvis_prepared
pyLDAvis.display(LDAvis_prepared)

c:\Users\mojea\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyLDAvis\_prepare.py:244: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)
